In [5]:
import os
import shutil
import random
from tqdm import tqdm
from ultralytics import YOLO

In [6]:
# Function to copy files
def copy_files(files, split, image_dir, label_dir):
    print('Copying files to', os.path.join(split))
    for file in tqdm(files):
        shutil.copy(os.path.join(image_dir, file + '.jpg'), os.path.join(split, 'images'))
        shutil.copy(os.path.join(label_dir, file + '.txt'), os.path.join(split, 'labels'))

def create_splits(image_dir, label_dir, train_pct, val_pct, max_samples=None):
    # Make sure the percentages add up to 100
    assert train_pct + val_pct <= 1.0, "Train and validation percentages should sum up to 1.0 or less"

    # Create directories for the splits if they don't exist
    for split in ['train', 'val', 'test']:
        for sub_dir in ['images', 'labels']:
            os.makedirs(os.path.join(split, sub_dir), exist_ok=True)

    # Gather all file names (without extensions)
    file_names = [os.path.splitext(file)[0] for file in os.listdir(image_dir)]

    # Shuffle the file names
    random.shuffle(file_names)

    # If max_samples is set, truncate the list
    if max_samples is not None:
        file_names = file_names[:max_samples]

    # Calculate split sizes
    total_files = len(file_names)
    train_size = int(total_files * train_pct)
    val_size = int(total_files * val_pct)

    # Split the file names
    train_files = file_names[:train_size]
    val_files = file_names[train_size:train_size + val_size]
    test_files = file_names[train_size + val_size:]


    # Copy files to respective directories
    copy_files(train_files, 'train', image_dir, label_dir)
    copy_files(val_files, 'val', image_dir, label_dir)
    copy_files(test_files, 'test', image_dir, label_dir)

    print(f"\nDataset split complete: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test samples.")

In [7]:
# Create the dataset for the correct task (segmentation or detection)
task = "detection"
create_splits('images', task, train_pct=0.7, val_pct=0.2)

Copying files to train


100%|██████████| 16382/16382 [00:02<00:00, 6780.41it/s]


Copying files to val


100%|██████████| 4680/4680 [00:00<00:00, 6801.29it/s]


Copying files to test


100%|██████████| 2342/2342 [00:00<00:00, 6583.84it/s]


Dataset split complete: 16382 train, 4680 val, 2342 test samples.


In [13]:
# load a pretrained COCO model and fine tune it (recommended for training)
model = YOLO('yolov8n.pt') # Detection

# Train the model
results = model.train(data='data.yaml', epochs=20, imgsz=640)

New https://pypi.org/project/ultralytics/8.2.96 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.95 🚀 Python-3.12.5 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

100%|██████████| 755k/755k [00:00<00:00, 31.0MB/s]

Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 225 layers, 3,013,383 parameters, 3,013,367 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: maxglo. Use `wandb login --relogin` to force relogin


...

      20/20      2.47G      1.053     0.7607      1.006        130        640: 100%|██████████| 1024/1024 [00:37<00:00, 27.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 147/147 [00:06<00:00, 23.71it/s]


                   all       4680      27382      0.821      0.645      0.726      0.534

20 epochs completed in 0.248 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.2.95 🚀 Python-3.12.5 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
Model summary (fused): 168 layers, 3,008,183 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 147/147 [00:07<00:00, 19.69it/s]


                   all       4680      27382      0.823      0.644      0.726      0.534
                Worker       3730      18144      0.869      0.655      0.771      0.477
          Static crane        797       1665      0.804      0.709      0.781      0.588
          Hanging head        434        576       0.81      0.326      0.418      0.214
                 Crane        435        579      0.795      0.646      0.716      0.517
                Roller        159        189      0.885      0.783      0.857      0.741
             Bulldozer        224        247      0.848      0.746      0.828       0.66
             Excavator       1720       2515      0.907      0.784      0.866      0.688
                 Truck        707       1187      0.806       0.57       0.68       0.51
                Loader        209        219      0.795      0.626      0.704      0.562
            Pump truck        287        321        0.8      0.769      0.802       0.62
        Concrete mixe

lr/pg0,▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
lr/pg1,▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
lr/pg2,▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
metrics/mAP50(B),▁▂▃▄▅▅▆▆▆▇▇▇▇▇██████
metrics/mAP50-95(B),▁▂▃▄▄▅▅▆▆▆▆▇▇▇▇▇████
metrics/precision(B),▁▃▃▅▅▆▆▇▆▇▇▇▇██▇████
metrics/recall(B),▁▂▃▄▅▅▅▅▆▆▇▇▇▇▇█████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▇▇▆▆▅▅▅▄▄▃▃▃▂▂▂▁▁▁
